# Chapter 19: Virtual Environments and Dependency Management

This notebook covers Python virtual environments -- the essential mechanism for isolating
project dependencies and ensuring reproducible builds. We explore the built-in `venv` module,
pip fundamentals, dependency resolution, and best practices for professional Python development.

## Key Concepts
- **Isolation**: Each project gets its own set of installed packages
- **Reproducibility**: Pin exact versions so builds are repeatable
- **venv module**: The standard library tool for creating virtual environments
- **pip**: The package installer for Python

## Section 1: Why Virtual Environments?

Without virtual environments, every Python project on your system shares the same
global `site-packages` directory. This causes real problems:

| Problem | Example |
|---|---|
| **Version conflicts** | Project A needs `requests==2.28` but Project B needs `requests==2.31` |
| **Polluted global state** | Installing packages globally can break system tools |
| **Non-reproducible builds** | No way to know which packages a project actually requires |
| **Permission issues** | System Python often requires `sudo` to install packages |

A virtual environment creates a **self-contained directory tree** with its own Python
interpreter and `site-packages`. Packages installed inside the environment are invisible
to other environments.

In [ ]:
import sys

# These paths reveal which Python environment is currently active.
# When running inside a virtual environment, sys.prefix differs from sys.base_prefix.

print(f"sys.executable:  {sys.executable}")
print(f"sys.prefix:      {sys.prefix}")
print(f"sys.base_prefix: {sys.base_prefix}")
print()

# The definitive check for whether you are inside a virtual environment
in_venv: bool = sys.prefix != sys.base_prefix
print(f"Inside a virtual environment: {in_venv}")

## Section 2: Creating Virtual Environments with `venv`

The `venv` module is part of the standard library (since Python 3.3). The typical
shell workflow is:

```bash
# Create a new virtual environment
python -m venv .venv

# Activate it (macOS / Linux)
source .venv/bin/activate

# Activate it (Windows)
.venv\Scripts\activate

# Deactivate when done
deactivate
```

The `.venv` directory contains:
- A copy or symlink to the Python interpreter
- A `lib/pythonX.Y/site-packages/` directory for installed packages
- Activation scripts that adjust `PATH` and `VIRTUAL_ENV`

**Convention**: Name the directory `.venv` (with a leading dot) so it is hidden on
Unix systems and easily matched by `.gitignore`.

In [ ]:
import os
import sys
import sysconfig

# Explore the directory structure that venv creates.
# sysconfig.get_paths() shows where packages and scripts are installed.

paths: dict[str, str] = sysconfig.get_paths()

print("Key installation paths:")
print(f"  purelib (pure Python packages): {paths['purelib']}")
print(f"  scripts (console scripts):      {paths['scripts']}")
print(f"  include (C headers):            {paths['include']}")
print()

# The VIRTUAL_ENV environment variable is set by the activation script
venv_path: str | None = os.environ.get("VIRTUAL_ENV")
print(f"VIRTUAL_ENV env var: {venv_path or '(not set -- not activated)'}")

## Section 3: Programmatic Environment Creation with `venv.EnvBuilder`

For tooling, CI pipelines, or scripts that need to create environments on the fly,
`venv.EnvBuilder` provides a Python API instead of shelling out to `python -m venv`.

In [ ]:
import venv
import tempfile
from pathlib import Path

# EnvBuilder accepts the same options as the CLI
builder = venv.EnvBuilder(
    system_site_packages=False,  # Do NOT inherit global packages
    clear=True,                  # Remove existing env dir first
    with_pip=True,               # Bootstrap pip into the new env
    symlinks=True,               # Use symlinks instead of copies (faster)
    upgrade_deps=False,          # Set True to upgrade pip/setuptools
)

# Create a temporary environment for demonstration
tmp_dir: Path = Path(tempfile.mkdtemp()) / "demo_env"
builder.create(str(tmp_dir))

print(f"Environment created at: {tmp_dir}")
print(f"\nDirectory contents:")
for item in sorted(tmp_dir.iterdir()):
    print(f"  {item.name}/" if item.is_dir() else f"  {item.name}")

In [ ]:
# Inspect the created environment more deeply
from pathlib import Path
import sys

# The pyvenv.cfg file stores the environment's configuration
cfg_file: Path = tmp_dir / "pyvenv.cfg"
if cfg_file.exists():
    print("Contents of pyvenv.cfg:")
    print(cfg_file.read_text())

# Find the Python executable inside the new environment
if sys.platform == "win32":
    python_path: Path = tmp_dir / "Scripts" / "python.exe"
else:
    python_path: Path = tmp_dir / "bin" / "python"

print(f"Python executable exists: {python_path.exists()}")
print(f"Python executable path:   {python_path}")

## Section 4: pip Basics -- Installing and Managing Packages

Once inside a virtual environment, `pip` is the standard tool for installing packages.

### Common Commands

```bash
# Install a package (latest version)
pip install requests

# Install a specific version
pip install requests==2.31.0

# Install with version constraints
pip install "requests>=2.28,<3.0"

# Install from a requirements file
pip install -r requirements.txt

# List installed packages
pip list

# Freeze current environment to a requirements file
pip freeze > requirements.txt

# Uninstall a package
pip uninstall requests
```

### `pip freeze` vs `pip list`

| Command | Output | Purpose |
|---|---|---|
| `pip list` | Human-readable table | Quick inspection |
| `pip freeze` | `package==version` lines | Machine-readable, for `requirements.txt` |

In [ ]:
import subprocess
import sys

# You can invoke pip programmatically via subprocess.
# IMPORTANT: Never use `import pip` directly -- it is not a public API.

result = subprocess.run(
    [sys.executable, "-m", "pip", "list", "--format=json"],
    capture_output=True,
    text=True,
)

import json

packages: list[dict[str, str]] = json.loads(result.stdout)
print(f"Installed packages in this environment: {len(packages)}")
print()

# Show first 10 packages
for pkg in packages[:10]:
    print(f"  {pkg['name']:30s} {pkg['version']}")

if len(packages) > 10:
    print(f"  ... and {len(packages) - 10} more")

## Section 5: Requirements Files and Pinning

A `requirements.txt` file lists the packages (and their versions) your project needs.
There are two common strategies:

### 1. Abstract requirements (flexible)
```
requests>=2.28
click~=8.0
pydantic<3
```
Good for **libraries** -- allows downstream users to resolve compatible versions.

### 2. Pinned requirements (exact)
```
requests==2.31.0
click==8.1.7
pydantic==2.5.3
```
Good for **applications** -- ensures identical installs across machines.

In [ ]:
from dataclasses import dataclass


@dataclass
class Requirement:
    """Represents a single line in a requirements file."""
    name: str
    specifier: str  # e.g. ">=2.28", "==2.31.0", "~=8.0"

    def __str__(self) -> str:
        return f"{self.name}{self.specifier}"


def parse_requirements(text: str) -> list[Requirement]:
    """Parse a simple requirements.txt into Requirement objects."""
    requirements: list[Requirement] = []
    for line in text.strip().splitlines():
        line = line.strip()
        if not line or line.startswith("#"):
            continue
        # Split at the first version specifier character
        for i, char in enumerate(line):
            if char in (">", "<", "=", "~", "!"):
                requirements.append(Requirement(line[:i], line[i:]))
                break
        else:
            requirements.append(Requirement(line, ""))
    return requirements


sample_requirements = """
# Web framework and utilities
flask>=3.0,<4.0
requests==2.31.0
click~=8.0
pydantic>=2.0
"""

reqs: list[Requirement] = parse_requirements(sample_requirements)
for req in reqs:
    print(f"  Package: {req.name:15s}  Specifier: {req.specifier}")

## Section 6: Dependency Resolution and Conflicts

When you install a package, pip must resolve its **transitive dependencies** -- the
packages your dependencies themselves depend on.

### How Conflicts Arise

```
your-project
  ├── package-A  requires  shared-lib>=2.0,<3.0
  └── package-B  requires  shared-lib>=1.0,<2.0
```

Here, `package-A` and `package-B` need **incompatible** versions of `shared-lib`.
pip will report a `ResolutionImpossible` error.

### Strategies for Resolving Conflicts
1. **Upgrade/downgrade** one of the conflicting packages
2. **Find alternative packages** that have compatible dependency trees
3. **Use `pip install --dry-run`** to preview what would be installed
4. **Use `pip check`** to verify the current environment is consistent

In [ ]:
import subprocess
import sys

# pip check verifies that all installed packages have compatible dependencies
result = subprocess.run(
    [sys.executable, "-m", "pip", "check"],
    capture_output=True,
    text=True,
)

print("pip check output:")
if result.stdout.strip():
    print(result.stdout)
else:
    print("  No broken requirements found.")

print(f"Return code: {result.returncode} (0 = all OK)")

In [ ]:
# Simulating dependency resolution logic
from dataclasses import dataclass, field


@dataclass
class VersionRange:
    """A simplified version constraint."""
    min_version: tuple[int, ...]
    max_version: tuple[int, ...]

    def contains(self, version: tuple[int, ...]) -> bool:
        return self.min_version <= version < self.max_version

    def overlaps(self, other: "VersionRange") -> bool:
        """Check if two version ranges have any overlap."""
        return self.min_version < other.max_version and other.min_version < self.max_version

    def __repr__(self) -> str:
        min_str = ".".join(str(x) for x in self.min_version)
        max_str = ".".join(str(x) for x in self.max_version)
        return f">={min_str},<{max_str}"


# Two packages require different versions of a shared dependency
constraint_a = VersionRange(min_version=(2, 0, 0), max_version=(3, 0, 0))
constraint_b = VersionRange(min_version=(1, 0, 0), max_version=(2, 0, 0))
constraint_c = VersionRange(min_version=(2, 5, 0), max_version=(4, 0, 0))

print(f"Constraint A: {constraint_a}")
print(f"Constraint B: {constraint_b}")
print(f"Constraint C: {constraint_c}")
print()
print(f"A overlaps B: {constraint_a.overlaps(constraint_b)}  (conflict!)")
print(f"A overlaps C: {constraint_a.overlaps(constraint_c)}  (compatible)")
print()

# Check if a specific version satisfies both A and C
test_version: tuple[int, ...] = (2, 7, 0)
print(f"Version 2.7.0 satisfies A: {constraint_a.contains(test_version)}")
print(f"Version 2.7.0 satisfies C: {constraint_c.contains(test_version)}")

## Section 7: Inspecting the Current Environment

Several standard library modules help you introspect the running Python environment.
This is invaluable for debugging "works on my machine" issues.

In [ ]:
import sys
import os
import site

# sys module: interpreter-level information
print("=== sys module ===")
print(f"Python version:    {sys.version}")
print(f"Platform:          {sys.platform}")
print(f"Executable:        {sys.executable}")
print(f"Prefix:            {sys.prefix}")
print(f"Base prefix:       {sys.base_prefix}")
print(f"In virtual env:    {sys.prefix != sys.base_prefix}")
print()

# site module: package installation locations
print("=== site module ===")
user_site: str | None = site.getusersitepackages()
global_sites: list[str] = site.getsitepackages()
print(f"User site-packages:   {user_site}")
print(f"Global site-packages: {global_sites}")

In [ ]:
import sys
from pathlib import Path

# sys.path determines where Python looks for importable modules.
# In a virtual environment, the env's site-packages is at the front.

print("Module search path (sys.path):")
for i, path in enumerate(sys.path):
    label = ""
    if "site-packages" in path:
        label = "  <-- site-packages"
    elif path == "":
        label = "  <-- current directory"
    print(f"  [{i:2d}] {path}{label}")

## Section 8: Best Practices

### One environment per project
Never share a virtual environment between unrelated projects. Each project should
have its own `.venv` directory at the project root.

### Always `.gitignore` the environment directory
```gitignore
# Virtual environments
.venv/
venv/
env/
```

### Pin your dependencies
For applications, use `pip freeze > requirements.txt` or better yet, a lock file
from a tool like `pip-tools`, `Poetry`, or `PDM`.

### Use `python -m pip` instead of bare `pip`
This guarantees you are using pip from the correct Python installation:
```bash
python -m pip install requests   # Always correct
pip install requests             # May point to wrong Python
```

### Automate environment setup
Provide a `Makefile` or script so teammates can get started quickly:
```makefile
venv:
	python -m venv .venv
	.venv/bin/pip install -r requirements.txt
```

In [ ]:
import shutil

# Clean up the temporary environment we created earlier
if tmp_dir.exists():
    shutil.rmtree(tmp_dir)
    print(f"Cleaned up temporary environment: {tmp_dir}")

# Demonstrate a helper function for environment setup
def create_project_env(
    project_dir: Path,
    env_name: str = ".venv",
    with_pip: bool = True,
    upgrade_deps: bool = True,
) -> Path:
    """Create a virtual environment for a project directory.

    Args:
        project_dir: The root directory of the project.
        env_name: Name of the environment directory.
        with_pip: Whether to install pip in the environment.
        upgrade_deps: Whether to upgrade pip and setuptools.

    Returns:
        Path to the created environment directory.
    """
    env_path: Path = project_dir / env_name
    builder = venv.EnvBuilder(
        system_site_packages=False,
        clear=True,
        with_pip=with_pip,
        symlinks=True,
        upgrade_deps=upgrade_deps,
    )
    builder.create(str(env_path))
    return env_path


print("\ncreate_project_env() is ready to use.")
print("Usage: create_project_env(Path('/path/to/project'))")

## Summary

### Virtual Environments
- **Purpose**: Isolate project dependencies from the system Python and other projects
- **`python -m venv .venv`**: Creates a lightweight virtual environment
- **`sys.prefix != sys.base_prefix`**: Definitive check for active virtual environment
- **`venv.EnvBuilder`**: Programmatic API for creating environments in scripts and tools

### pip and Dependencies
- **`pip install`**: Install packages; use version specifiers for control
- **`pip freeze`**: Export exact versions for reproducibility
- **`pip check`**: Verify dependency consistency in the current environment
- **Conflicts**: Arise when transitive dependencies require incompatible versions

### Best Practices
- One `.venv` per project, always in `.gitignore`
- Use `python -m pip` to avoid PATH ambiguity
- Pin dependencies for applications; use flexible specifiers for libraries
- Automate environment creation with Makefiles or setup scripts